# Формирования таблицы словаря 'Marks' для базы данных.

### 1. Подготовка

In [21]:

import os
import time
import json
import datetime

import dateutil
import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine


### 2. Формирую переменные с нужными индексами марки и сотрудников, для формирования нужных тендеров.

Для этого подтягиваю из временных таблиц сформированных в других скриптах нужные данные.
Данные подтягиваю от даты начала предыдущего года.

In [ ]:

engine_pet = create_engine('mysql+mysqlconnector://root:''@''/pet_proect')

df_marks_ = pd.read_sql(
    'SELECT * FROM tmp_dict_marks', 
    con = engine_pet, 
    index_col='_id'
).index.to_list()

df_users_ = pd.read_sql(
    'SELECT * FROM tmp_dict_users', 
    con = engine_pet, 
    index_col = '_id'
).index.to_list()

df_marks = '&marks=' + '&marks='.join(df_marks_)
df_users = '&users=' + '&users='.join(df_users_)

date_first = (
    datetime.datetime.today() + 
    dateutil.relativedelta.relativedelta(
        years =- 1, 
        month = 1, 
        day = 1, 
        hour = 0, 
        minute = 0, 
        seconds = 0, 
        microseconds = 0
    )
)

unix_date_first = int(date_first.timestamp() * 1000)


### 3. Формирование переменных для подключения

In [ ]:

TOKEN = 'Bearer '
API = '/api/tenders/v2/getlist'
URL = 'https://tenderplan.ru' + API
headers = {
    'Authorization': TOKEN,
    'Accept': 'application/json'
}


### 4. Формирование функции get запроса для цилка и функции преобразования времени

In [4]:

if __name__ == '__main__':
    def f_lst_tender(page_):
        '''Функция принимает в качестве аргумента номер страницы и выводит данные get запроса'''
        response = rq.get(
            URL + 
            f'?page={str(page_)}' + 
            f'&fromPublicationDateTime={str(unix_date_first)}' + 
            df_marks + 
            df_users,
            headers = headers
        ).json()
        return response['tenders']

def f_date_fr_time(x):
    '''Принимает UNIX-время и возвращате дату в формате datetime'''
    return datetime.datetime.fromtimestamp(x//1000)

def round_2(x):
    '''Функция округления числа'''
    return round(x, 2)


### 5. Загрузка данных через цикл WHILE (проверка на пустую страницу). 

На каждой итерации таймаут 0,08 секунд, так как в документации к API ограничение на кол-во запросов. Не более 250 запросов за 10 секунд или не более 800 запросов за 60 секунд

In [5]:

lst_df = []
start_time = time.time()
count_page = 0

while f_lst_tender(count_page) != []:
    end_time = time.time()
    df_lst_tenders = f_lst_tender(count_page)
    lst_df = lst_df + df_lst_tenders
    count_page+=1
    time.sleep(0.08)
    
end_time = time.time()
del df_lst_tenders

print(f'Кол-во секунд: {round(end_time - start_time, 0)}')
print(f'Кол-во запросов: {len(lst_df)/50}')
print(f'Кол-во запросов в секунду: {len(lst_df)/50/(end_time - start_time)}')
print(f'Кол-во секунд на запрос: {(end_time - start_time)/(len(lst_df)/50)}')


Кол-во секунд: 95.0
Кол-во запросов: 115.9
Кол-во запросов в секунду: 1.2222989997727638
Кол-во секунд на запрос: 0.818130424868145


### 6. На данном этапе формируется несколько DataFrame:
1. df_participans - DataFrame участников тендера
2. df_marks_users - таблица фактов с пересечением марки, сотрудника и id тендера (несколько марок может быть на один тендер)
3. df_dict_tenders - справочник тенедров, были cJoinнены данные по стат показателям (мин, макс, ср, медиана по участникам тендера)

In [7]:

'''Создание всех DataFrame'''

'''Создание DataFrame участников'''
df_participants = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id', 
            'participants'
        ]
    )
    .set_index('_id')
    .explode('participants')
    .dropna(subset = 'participants')
)

df_participants = (
    pd.json_normalize(df_participants['participants'])
    .set_index(df_participants.index)
    .loc[
        :, 
        [
            'guid', 
            'name', 
            'price', 
            'region', 
            'score', 
            'winner'
        ]
    ]
)
    
df_participants = (
    df_participants
    .assign(
        **{
            'price': df_participants['price'].map(round_2),
            'score': df_participants['score'].map(round_2)
        }
    )
    .fillna(
        {
            'winner': np.nan,
            'guid': np.nan
        }
    )
)

'''Создание DataFrame стат. данных по участникам для справочника тендеров'''

agg_price_score = {
    'price': ['mean', 'std', 'min', 'max'],
    'score': ['mean', 'std', 'min', 'max']
}

df_stat_participants = (
    df_participants
    .groupby('_id')[['price', 'score']]
    .agg(agg_price_score)
    .round(2)
)

'''Перименование стобцов'''
df_stat_participants.columns = ['-'.join(x) for x in df_stat_participants.columns.values]

'''Создание DataFrame данных заказчика для справочника тендеров'''

df_customers = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'customers'
        ]
    )
    .set_index('_id')
    .explode('customers')
)

df_customers = (
    pd.json_normalize(df_customers['customers'])
    .set_index(df_customers.index)
    .astype(str)
    .groupby('_id')[['guid', 'name', 'region']]
    .agg(' '.join)
)

'''Создание DataFrame тендеров с денормализванными данными по меткам и сотрудникам'''

df_marks_users = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'marks',
            'users'
        ]
    )
    .set_index('_id')
    .explode('marks')
    .explode('users')
)

df_marks_users = df_marks_users.loc[
    (df_marks_users['marks'].isin(df_marks_)) & 
    (df_marks_users['users'].isin(df_users_))
]

'''Создание итого справочника по тендерам с данными заказчика и стат данными по участникам'''

df_dict_tenders = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'maxPrice',
            'status',
            'placingWay',
            'orderName',
            'publicationDate',
            'number',
            'region'
        ]
    )
    .set_index('_id')
    .join(
        df_customers, 
        rsuffix = '/customres'
    )
    .join(df_stat_participants)
)

df_dict_tenders = df_dict_tenders.assign(
    **{
        'publicationDate': df_dict_tenders['publicationDate'].map(f_date_fr_time)
    }
)


### 7. Загрузка данных во временные таблицы tmp

In [8]:

(
    df_participants
    .reset_index()
    .to_sql(
        'tmp_participants', 
        con=engine_pet, 
        if_exists='replace', 
        index=False
    )
)
time.sleep(1)
(
    df_marks_users
    .reset_index()
    .to_sql(
        'tmp_marks_users', 
        con=engine_pet, 
        if_exists='replace', 
        index=False
    )
)
time.sleep(1)
(
    df_dict_tenders
    .reset_index()
    .to_sql(
        'tmp_dict_tenders', 
        con=engine_pet, 
        if_exists='replace', 
        index=False
    )
)


-1

In [9]:

del df_stat_participants
del df_customers
del agg_price_score
del df_marks_
del df_users_


In [10]:
df_participants.head()

,guid,name,price,region,score,winner
_id,,,,,,
69022f0e5d48d39bb4047848,591eb8d40640fd17cdf5bbd7,"ООО УПК ""АРМАКОМ""",111066.04,54.0,0.64,NaN
69022f0e5d48d39bb4047848,591f12210640fd1a867a7f9f,"ООО ""СК МАГНАТ""",76891.87,73.0,0.67,NaN
69022f0e5d48d39bb4047848,591f1d8e0640fd1a867aa8df,"ООО ""ЦЕНТР ОПТОВОЙ ТОРГОВЛИ""",93978.96,66.0,0.62,NaN
69022f0e5d48d39bb4047848,595c628a2567a56d7cc6c556,"ООО ""АКСИОМА СЕРВИС""",106794.27,64.0,0.62,NaN
69022f0e5d48d39bb4047848,596e160c2567a54708c7ad76,"ООО ""ЮНИТРЕЙД""",111066.04,63.0,0.64,NaN


In [11]:
df_marks_users.head()

,marks,users
_id,,
69022f0e5d48d39bb4047848,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
69022eb55d48d39bb4020a68,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
69022d6d5d48d39bb4fd269c,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
690221305d48d39bb497b9fb,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36
690212ff5d48d39bb43b7330,674d27e6fe9d01373b795741,674d2d897b024fbedd74be36


In [12]:
df_dict_tenders.head()

,maxPrice,status,placingWay,orderName,publicationDate,number,region,guid,name,region/customres,price-mean,price-std,price-min,price-max,score-mean,score-std,score-min,score-max
_id,,,,,,,,,,,,,,,,,,
69022f0e5d48d39bb4047848,123881.35,1,22,Поставка запорной арматуры в рамках поддержани...,2025-10-29 07:00:00,0373100075525000037,77,58c7deb90640fd10b191dfed,ФКУ СИЗО-7 ГУФСИН РОССИИ ПО Г. МОСКВЕ,77,103804.03,14103.32,76891.87,119609.58,0.64,0.02,0.62,0.67
69022eb55d48d39bb4020a68,568156.83,1,22,Поставка сантехматериалов,2025-10-29 07:00:00,0372100049625003055,78,58c7ceb70640fd0b5eaacbad,"ФГБУ ""НМИЦ ИМ. В.А. АЛМАЗОВА"" МИНЗДРАВА РОССИИ",78,511341.15,44723.46,431015.53,548565.22,0.37,0.05,0.30,0.46
69022d6d5d48d39bb4fd269c,1814111.60,1,15,Поставка шарового крана для нужд ГБУЗ «ММКЦ «К...,2025-10-29 07:00:00,0373200087825002316,77,62bbaf03fc56c96dbd86b171,"ГБУЗ ""ММКЦ ""КОММУНАРКА"" ДЗМ""",77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690221305d48d39bb497b9fb,27933.68,1,15,Поставка сантехнических материалов,2025-10-29 07:00:00,0359100003725000096,62,63f01c4cfa511792804a115a,САСОВСКОЕ ИМЕНИ ГЕРОЯ СОВЕТСКОГО СОЮЗА ТАРАНА ...,62,20998.42,3874.27,14448.46,26970.45,0.46,0.11,0.38,0.63
690212ff5d48d39bb43b7330,21681.00,1,22,поставка Радиаторы центрального отопления,2025-10-29 07:00:00,0358200014625000221,61,58c7daee0640fd10a239e11c,"ГБУСОН РО ""РЦ ГОРОДА ЗВЕРЕВО""",61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
